In [2]:
import torch
import torch.nn as nn

import datasets
import models.resnet as ResNet
import models.senet as SENet
from trainer import Trainer, Validator
from extractor import Extractor
import utils

In [3]:
# 定义是否使用GPU
cuda = torch.cuda.is_available()
if cuda:
    print("torch.backends.cudnn.version: {}".format(torch.backends.cudnn.version()))

    torch.manual_seed(1337)
if cuda:
    torch.cuda.manual_seed(1337)


torch.backends.cudnn.version: 7005


In [4]:
# 超参数设置
configurations = {
    1: dict(
        max_iteration=1000000,
        lr=1.0e-1,
        momentum=0.9,
        weight_decay=0.0,
        gamma=0.1, # "lr_policy: step"
        step_size=1000000, # "lr_policy: step"
        interval_validate=1000,
    ),
}

In [5]:
# get parameters of each layer
def get_parameters(model, bias=False):
    for k, m in model._modules.items():
        if k == "fc" and isinstance(m, nn.Linear):
            if bias:
                yield m.bias
            else:
                yield m.weight

N_IDENTITY = 8631  # the number of identities in VGGFace2 for which ResNet and SENet are trained

In [6]:
utils.create_dir('checkpoint_directory')
cfg = configurations[1]
log_file = 'log_file'
resume = ''

In [7]:
# 0. id label map
meta_file = 'identity_meta.csv'
id_label_dict = utils.get_id_label_map(meta_file)

C:\Users\s439476\Documents\VGGFace2-pytorch-master\utils.py:37: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(identity_list, sep=',\s+', quoting=csv.QUOTE_ALL, encoding="utf-8")


In [8]:
# 1. data loader
root = 'datasets'    
train_img_list_file = 'train_list.txt'
test_img_list_file = 'test_list.txt'
kwargs = {'num_workers': 4, 'pin_memory': True} if cuda else {}

In [9]:
# 准备数据集并预处理
bs = 16#16#32
dt = datasets.VGG_Faces2(root, train_img_list_file, id_label_dict, split='train')
train_loader = torch.utils.data.DataLoader(dt, batch_size=bs, shuffle=True, **kwargs)

dv = datasets.VGG_Faces2(root, test_img_list_file, id_label_dict, split='test',
                             horizontal_flip='store_true')
val_loader = torch.utils.data.DataLoader(dv, batch_size=bs, shuffle=False, **kwargs)

processing: 0 images for train
processing: 1000 images for train
processing: 2000 images for train
processing: 3000 images for train
processing: 4000 images for train
processing: 5000 images for train
processing: 6000 images for train
processing: 7000 images for train
processing: 8000 images for train
processing: 9000 images for train
processing: 10000 images for train
processing: 11000 images for train
processing: 12000 images for train
processing: 13000 images for train
processing: 14000 images for train
processing: 15000 images for train
processing: 16000 images for train
processing: 17000 images for train
processing: 18000 images for train
processing: 19000 images for train
processing: 20000 images for train
processing: 21000 images for train
processing: 22000 images for train
processing: 23000 images for train
processing: 24000 images for train
processing: 25000 images for train
processing: 26000 images for train
processing: 27000 images for train
processing: 28000 images for trai

processing: 457000 images for train
processing: 458000 images for train
processing: 459000 images for train
processing: 460000 images for train
processing: 461000 images for train
processing: 462000 images for train
processing: 463000 images for train
processing: 464000 images for train
processing: 465000 images for train
processing: 466000 images for train
processing: 467000 images for train
processing: 468000 images for train
processing: 469000 images for train
processing: 470000 images for train
processing: 471000 images for train
processing: 472000 images for train
processing: 473000 images for train
processing: 474000 images for train
processing: 475000 images for train
processing: 476000 images for train
processing: 477000 images for train
processing: 478000 images for train
processing: 479000 images for train
processing: 480000 images for train
processing: 481000 images for train
processing: 482000 images for train
processing: 483000 images for train
processing: 484000 images fo

processing: 911000 images for train
processing: 912000 images for train
processing: 913000 images for train
processing: 914000 images for train
processing: 915000 images for train
processing: 916000 images for train
processing: 917000 images for train
processing: 918000 images for train
processing: 919000 images for train
processing: 920000 images for train
processing: 921000 images for train
processing: 922000 images for train
processing: 923000 images for train
processing: 924000 images for train
processing: 925000 images for train
processing: 926000 images for train
processing: 927000 images for train
processing: 928000 images for train
processing: 929000 images for train
processing: 930000 images for train
processing: 931000 images for train
processing: 932000 images for train
processing: 933000 images for train
processing: 934000 images for train
processing: 935000 images for train
processing: 936000 images for train
processing: 937000 images for train
processing: 938000 images fo

processing: 1355000 images for train
processing: 1356000 images for train
processing: 1357000 images for train
processing: 1358000 images for train
processing: 1359000 images for train
processing: 1360000 images for train
processing: 1361000 images for train
processing: 1362000 images for train
processing: 1363000 images for train
processing: 1364000 images for train
processing: 1365000 images for train
processing: 1366000 images for train
processing: 1367000 images for train
processing: 1368000 images for train
processing: 1369000 images for train
processing: 1370000 images for train
processing: 1371000 images for train
processing: 1372000 images for train
processing: 1373000 images for train
processing: 1374000 images for train
processing: 1375000 images for train
processing: 1376000 images for train
processing: 1377000 images for train
processing: 1378000 images for train
processing: 1379000 images for train
processing: 1380000 images for train
processing: 1381000 images for train
p

processing: 1797000 images for train
processing: 1798000 images for train
processing: 1799000 images for train
processing: 1800000 images for train
processing: 1801000 images for train
processing: 1802000 images for train
processing: 1803000 images for train
processing: 1804000 images for train
processing: 1805000 images for train
processing: 1806000 images for train
processing: 1807000 images for train
processing: 1808000 images for train
processing: 1809000 images for train
processing: 1810000 images for train
processing: 1811000 images for train
processing: 1812000 images for train
processing: 1813000 images for train
processing: 1814000 images for train
processing: 1815000 images for train
processing: 1816000 images for train
processing: 1817000 images for train
processing: 1818000 images for train
processing: 1819000 images for train
processing: 1820000 images for train
processing: 1821000 images for train
processing: 1822000 images for train
processing: 1823000 images for train
p

processing: 2239000 images for train
processing: 2240000 images for train
processing: 2241000 images for train
processing: 2242000 images for train
processing: 2243000 images for train
processing: 2244000 images for train
processing: 2245000 images for train
processing: 2246000 images for train
processing: 2247000 images for train
processing: 2248000 images for train
processing: 2249000 images for train
processing: 2250000 images for train
processing: 2251000 images for train
processing: 2252000 images for train
processing: 2253000 images for train
processing: 2254000 images for train
processing: 2255000 images for train
processing: 2256000 images for train
processing: 2257000 images for train
processing: 2258000 images for train
processing: 2259000 images for train
processing: 2260000 images for train
processing: 2261000 images for train
processing: 2262000 images for train
processing: 2263000 images for train
processing: 2264000 images for train
processing: 2265000 images for train
p

processing: 2681000 images for train
processing: 2682000 images for train
processing: 2683000 images for train
processing: 2684000 images for train
processing: 2685000 images for train
processing: 2686000 images for train
processing: 2687000 images for train
processing: 2688000 images for train
processing: 2689000 images for train
processing: 2690000 images for train
processing: 2691000 images for train
processing: 2692000 images for train
processing: 2693000 images for train
processing: 2694000 images for train
processing: 2695000 images for train
processing: 2696000 images for train
processing: 2697000 images for train
processing: 2698000 images for train
processing: 2699000 images for train
processing: 2700000 images for train
processing: 2701000 images for train
processing: 2702000 images for train
processing: 2703000 images for train
processing: 2704000 images for train
processing: 2705000 images for train
processing: 2706000 images for train
processing: 2707000 images for train
p

processing: 3123000 images for train
processing: 3124000 images for train
processing: 3125000 images for train
processing: 3126000 images for train
processing: 3127000 images for train
processing: 3128000 images for train
processing: 3129000 images for train
processing: 3130000 images for train
processing: 3131000 images for train
processing: 3132000 images for train
processing: 3133000 images for train
processing: 3134000 images for train
processing: 3135000 images for train
processing: 3136000 images for train
processing: 3137000 images for train
processing: 3138000 images for train
processing: 3139000 images for train
processing: 3140000 images for train
processing: 3141000 images for train
processing: 0 images for test
processing: 1000 images for test
processing: 2000 images for test
processing: 3000 images for test
processing: 4000 images for test
processing: 5000 images for test
processing: 6000 images for test
processing: 7000 images for test
processing: 8000 images for test
pro

In [10]:
# 2. model
include_top = True
model = ResNet.resnet50(num_classes=N_IDENTITY, include_top=include_top)

In [11]:
# 超参数设置
start_epoch = 0
start_iteration = 0
weight_file = 'resnet50_ft_weight.pkl'
utils.load_state_dict(model, weight_file)
model.fc.reset_parameters()

In [12]:
# 定义损失函数
if cuda:
    model = model.cuda()
    
criterion = nn.CrossEntropyLoss()
if cuda:
    criterion = criterion.cuda()

In [13]:
# 3. optimizer
optim = torch.optim.SGD(
    [
        {'params': get_parameters(model, bias=False)},
        {'params': get_parameters(model, bias=True), 'lr': cfg['lr'] * 2, 'weight_decay': 0},
    ],
    lr=cfg['lr'],
    momentum=cfg['momentum'],
    weight_decay=cfg['weight_decay'])

In [14]:
last_epoch = start_iteration if resume else -1
lr_scheduler = torch.optim.lr_scheduler.StepLR(optim,  cfg['step_size'],
                           gamma=cfg['gamma'], last_epoch=last_epoch)

In [15]:
#call the trainer
trainer = Trainer(
    cmd='train',
    cuda=cuda,
    model=model,
    criterion=criterion,
    optimizer=optim,
    lr_scheduler=lr_scheduler,
    train_loader=train_loader,
    val_loader=val_loader,
    log_file=log_file,
    max_iter=cfg['max_iteration'],
    checkpoint_dir='checkpoint_directory',
    print_freq=1,
)
trainer.epoch = start_epoch
trainer.iteration = start_iteration

In [16]:
trainer.train()

Train epoch=0, iter=0:   0%|                         | 0/261825 [00:00<?, ?it/s]

Train: [0/261825/12]	epoch: 0	iter: 0	Time: 29.265 (29.265)	Data: 27.615 (27.615)	Loss: 9.5512 (9.5512)	Prec@1: 0.000 (0.000)	Prec@5: 0.000 (0.000)	lr 0.100000



Train epoch=0, iter=0:   0%|             | 1/261825 [00:01<144:40:55,  1.99s/it]

Train: [1/261825/24]	epoch: 0	iter: 1	Time: 0.559 (14.912)	Data: 0.236 (13.925)	Loss: 32.6815 (21.1163)	Prec@1: 0.000 (0.000)	Prec@5: 0.000 (0.000)	lr 0.100000



Train epoch=0, iter=0:   0%|             | 2/261825 [00:02<114:46:14,  1.58s/it]

...

Train: [780/261825/9372]	epoch: 0	iter: 780	Time: 0.638 (0.733)	Data: 0.302 (0.399)	Loss: 266.4034 (289.6967)	Prec@1: 8.333 (3.852)	Prec@5: 16.667 (6.231)	lr 0.100000



Train epoch=0, iter=0:   0%|            | 781/261825 [09:05<46:21:00,  1.56it/s]

Train: [781/261825/9384]	epoch: 0	iter: 781	Time: 0.627 (0.733)	Data: 0.297 (0.399)	Loss: 300.7274 (289.7109)	Prec@1: 0.000 (3.847)	Prec@5: 8.333 (6.234)	lr 0.100000



Train epoch=0, iter=0:   0%|            | 782/261825 [09:06<46:25:09,  1.56it/s]

KeyboardInterrupt: 